# Interpreting Models

As machine learning becomes more and more and more prevelant, the predictions made by models have greater influence over many aspects of our society. For example, machine learning models are an increasingly significant factor in how banks decide to grant loans or doctors prioritise treatments. The ability to interpret and explain models is increasingly important, so that the rationale for the predictions made by machine learning models can be explained and justified, and any inadvertant bias in the model can be identified.

## Explaining a Model
You can use Azure Machine Learning to interpret a model by using an *explainer* that quantifies the amount of influence each feature contribues to the predicted label. There are many common explainers, each suitable for different kinds of modeling algorithm; but the basic approach to using them is the same.

> **Note**: For more information about explainers in Azure ML, see [the documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-machine-learning-interpretability). 

Let's start with a model that is trained outside of Azure Machine Learning - we'll go ahead and train a simple binary classification model that predicts the likelihood of a patient being diabetic.

In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
%matplotlib inline

# Set regularization rate hyperparameter value
reg = 0.1

# load the diabetes data
print("Loading Data...")
data = pd.read_csv('data/diabetes.csv')

features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']

# Separate features and labels
X, y = data[features].values, data['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

Our training process generated some model evaluation meytrics based on a hold-back validation dataset, so we have an idea of how accurately it predicts; but how do the features in the data influence the prediction?

### Install the Azure ML Interpretability Library
To find out, we'll first install the Azure ML Interpretability library. You can use this to interpret many typical kinds of model, even if they haven't been trained in an Azure ML experiment or registered in an Azure ML workspace.

In [ ]:
!pip install azureml-interpret

### Get an Explainer for our Model

Noe that we have the library installed, let's get a suitable explainer for our model. There are many kinds of explainer. In this example we'll use a *Mimic Explainer*, which is a "black box" explainer that can be used to explain many kinds of model. We'll specifically use it to explain a model that's been trained using a linear algorithm - such as our logistic regression model.

In [ ]:
from interpret.ext.blackbox import MimicExplainer
from interpret.ext.glassbox import LinearExplainableModel

# "features" and "classes" fields are optional
explainer = MimicExplainer(model, 
                           X_train, 
                           LinearExplainableModel, 
                           augment_data=True, 
                           max_num_of_augmentations=10, 
                           features=features, 
                           classes=labels)

print(explainer, "ready!")

### Get Global Feature Importance

The first thing we'll do is try to explain the model by evaluating the overall *feature importance* - in other words, quantifying the extent to which each feature influences the prediction based on the whole training dataset.

In [ ]:
# you can use the training data or the test data here
global_explanation = explainer.explain_global(X_train)

# Get the top features by importance
global_feature_importance = global_explanation.get_feature_importance_dict()
for feature, importance in global_feature_importance.items():
    print(feature,":", importance)


(you can ignore any warnings that are displayed)

The feature importance is ranked, with the most important feature listed first.

### Get Local Feature Importance

So we have an overall view, but what about explaining individual observations? Let's generate *local* explanations for individual predictions, quantifying the extent to which each feature influenced the decision to predict each of the possible label values. In this case, it's a binary model, so there are two possible labels (non-diabetic and diabetic); and we can quantify the influence of each feature for each of these label values for individual observations in a dataset. We'll just evaluate the first three cases in the test dataset.

In [ ]:
# Get the observations we want to explain (the first five)
X_explain = X_test[0:5]

# Get predictions
predictions = model.predict(X_explain)

# Get local explanations
local_explanation = explainer.explain_local(X_explain)

# Get feature names and importance for each possible label
local_features = local_explanation.get_ranked_local_names()
local_importance = local_explanation.get_ranked_local_values()

for l in range(len(local_features)):
    print('Support for', labels[l])
    label = local_features[l]
    for o in range(len(label)):
        print("\tObservation", o + 1)
        feature_list = label[o]
        total_support = 0
        for f in range(len(feature_list)):
            print("\t\t", feature_list[f], ':', local_importance[l][o][f])
            total_support += local_importance[l][o][f]
        print("\t\t ----------\n\t\t Total:", total_support, "Prediction:", labels[predictions[o]])



### Visualize Explanations

The Azure ML SDK also includes a notebook widget that you can use to visualize explanations.

Now you can use the widget to visualize the explanation you created from the model previously.

> **Note**: The explanation visualization widget is in preview, and may  crash your browser!

In [ ]:
from azureml.contrib.interpret.visualize import ExplanationDashboard

ExplanationDashboard(global_explanation, model, X_train)

## Adding Explainability to Azure ML Models Training Experiments

As you've seen, you can generate explanations for models trained ouytside of Azure ML; but when you use experiments to train models in your Azure ML workspace, you can generate model explanations and log them.

Let's first of all ensure you have the latest version of the SDK installed.

In [ ]:
!pip install --upgrade azureml-sdk[notebooks]
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

And now, let's connect to your workspace.

In [ ]:
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

### Train and Explain a Model using an Experiment

OK, let's create an experiment and put the files it needs in a local folder - in this case we'll just use the same CSV file of diabetes data to train the model.

In [ ]:
import os, shutil
from azureml.core import Experiment

# Create an experiment
experiment_name = 'diabetes_train_and_explain'
experiment = Experiment(workspace = ws, name = experiment_name)

# Create a folder for the experiment files
experiment_folder = './' + experiment_name
os.makedirs(experiment_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(experiment_name, "diabetes.csv"))

print("Experiment:", experiment.name)

Now we'll create a training script that looks similar to any other Azure ML training script except that is includes the following features:

- The same libraries to generate model explanations we used before are imported and used to generate a global explanation
- The **ExplanationClient** library is used to upload the epxlanation to the experiment output

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Import Azure ML run library
from azureml.core.run import Run

# Import libraries for model explanation
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient
from interpret.ext.blackbox import MimicExplainer
from interpret.ext.glassbox import LinearExplainableModel

# Set regularization hyperparameter (passed as an argument to the script)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
data = pd.read_csv('diabetes.csv')

features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']

# Separate features and labels
X, y = data[features].values, data['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
run.log('AUC', np.float(auc))

# log a ROC curve plot
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes.pkl')

# Get the model explanation and upload it
explain_client = ExplanationClient.from_run(run)
explainer = MimicExplainer(model, 
                           X_train, 
                           LinearExplainableModel, 
                           augment_data=True, 
                           max_num_of_augmentations=10, 
                           features=features, 
                           classes=labels)
global_explanation = explainer.explain_global(X_test)
explain_client.upload_model_explanation(global_explanation, comment='global explanation: all features')

run.complete()

Now you can run the experiment, using an estimator to run the training script. Note that the **azureml-contrib-interpret** package is included in the training environment.

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Set the script parameters
script_params = {
    '--regularization': 0.1
}

# Create a Python environment for the experiment
environment_name = "diabetes-env"
diabetes_env = Environment(environment_name)
diabetes_env.python.user_managed_dependencies = False 
diabetes_env.docker.enabled = False
diabetes_conda = CondaDependencies.create(conda_packages=['pandas','scikit-learn','joblib','ipykernel','matplotlib'],
                                          pip_packages=['azureml-sdk', 'azureml-contrib-interpret','argparse','pyarrow'])
diabetes_env.python.conda_dependencies = diabetes_conda
# Register the environment so we can re-use it later.
diabetes_env.register(ws)

# Create an estimator
estimator = Estimator(source_directory=experiment_folder,
              script_params=script_params,
              compute_target = 'local', # Use local compute
              environment_definition = Environment.get(ws, environment_name),
              entry_script='diabetes_training.py')

# Run the experiment
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

### View the Model Explanation
With the experiment run completed, you can use an **ExplanationClient** object to download it from the logged run, and view the feature importance data.

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

client = ExplanationClient.from_run(run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance (the Feature value is the column index in the training data)
print("Feature\tImportance")
for key, value in feature_importances.items():
    print(key, ":", value)

You can also view the model explanation in [Azure ML Studio](https://ml.azure.com). Just open the latest run of the experiment, and view the **Explanations** tab.